# 0. Create Quarto YAML Cell and Import Libraries

The cell given above must go at the very top of your Notebook, even before the title!  Be sure to include the `---` on the first and last lines.  No other content should go in this raw cell!  This is what it should look like:

```

---
title: "Encoding Music Sample Homework Project"
author: 'Richard Freedman'
date: 'June 25, 2024'

format:
  html:
    code-fold: true
    embed-resources: true
plotly-connected: true
jupyter: python3

---
```

##  Now the Title, Your Name, and Date:

You can use markdown to control the size and look of this information:

```
Music 255:  Encoding Music
My First Homework Project
Richard Freedman
September 2, 2024
```

And now the **Import** statements, where you load the relevant libraries.  

Note that the last to items (for Quarto and Warnings) are especially important.

In [1]:
# import libraries
from community import community_louvain
from copy import deepcopy
from IPython.display import SVG
import altair as alt
import glob as glob
import numpy as np
import os
import pandas as pd
import re
import networkx as nx
import plotly.express as px
import requests

# setup plotting for quarto
alt.renderers.enable('default')
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# supress warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Motivation

What is your point of inquiry? What are you trying discover? What do you expect to find? How do these questions relate to themes and problems considered in the course so far?

What specific readings or musical repertories are involved, and why do they matter in this context (be sure to cite them, and to follow the Music Style Guide as you do so)?

# 2. Implementation

How did you put your query into practice? What did you need to do with or to your data along the way?

* What logical steps did you follow as you thought about getting there (expressed in pseudo code; perhaps in bullet-point form in your markdown cell)?
* Which tools (or methods) did you select, and why were they right for the job?
If you chose to create some kind of visualization (a chart, network or map), how did you decide on the right representation for the task at hand?
* What challenges did you face as you worked to put the steps into action in code? Did you need to clean or tidy your data? Filter, sort, or group it in some way?
* What were the most important tools or approaches you deployed that others might like to know about?

Remember to use verbatim code or block code to show your work as needed. 

Remember to add comments to your code to show the most important steps.

If you learned from or adapted some code suggested in Encoding Music, or another resource, it would be good to cite that documentation, too.

**Note**:  Your code cells and their outputs go below. 

When you 'render' this with **Quarto** (see [instructions here](https://github.com/RichardFreedman/Encoding_Music/blob/main/01_Tutorials/01_Notebook_Introduction_Quarto.md#Export-and-Publish-with-Quarto)) the code will be 'hidden' behind a dropdown arrow.  The output will be rendered in HTML.

## a. Now Let's Import our Messy Data


In [2]:

# give the url a simple name
our_messy_metadata_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQi5q2j94W-HPrVsgtK0rsncYEu3ElLOk2p4MlEDW8vbBUXOEOKNNuN63Mohae062tmARINbpklDe-w/pub?gid=190053203&single=true&output=csv'

# load it as df
our_messy_metadata = pd.read_csv(our_messy_metadata_csv)

# view the head
our_messy_metadata.head()



,Timestamp,Song Title,Your Name,Your Graduating Class,Team/Group Name,Personal Rank,Genre Tags,Most Likely Context(s),Least Likely Contexts
0,9/17/2025 15:21:26,Blackbird,Natalie,Class o’ ‘28,"Phlurp, Purple",3rd Best,{soft rock} {indie rock} {rain},being_bored_in_class studying driving,watching_a_live_performance working_out
1,9/17/2025 15:22:00,Eleanor Rigby,cameron_neville,29,Greenmachine,4st,baroque.pop | alternative.rock,Walkin',Dancin'
2,9/17/2025 15:22:33,Yesterday,isabella_law,27,Greenmachine,1st,acoustic.rock | british.invasion | classic.rock,Cryin',Moshin'
3,9/17/2025 15:23:31,A Day in the Life,Maryam Imran Siddiqui,27,Orange Crush,3,Rock,walking/driving,studying
4,9/17/2025 15:23:54,Blackbird,Natalie,Class o’ ‘28,"Phlurp, Purple",4th Best,{emo rap} {plugg},walking driving studying,waiting_in_an_elevator


Everything here seems to be just an 'object' column.

- timestamp should be a date
- graduating class should be an integer (eventually)
- personal rank should be an integer


In [3]:

# info
our_messy_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Timestamp               66 non-null     object
 1   Song Title              66 non-null     object
 2   Your Name               66 non-null     object
 3   Your Graduating Class   66 non-null     object
 4   Team/Group Name         66 non-null     object
 5   Personal Rank           66 non-null     object
 6   Genre Tags              66 non-null     object
 7   Most Likely Context(s)  66 non-null     object
 8   Least Likely Contexts   66 non-null     object
dtypes: object(9)
memory usage: 4.8+ KB


### Ranks

In [4]:
rank_list = list(our_messy_metadata['Personal Rank'].unique())
rank_dict = {}
rank_dict = rank_dict.fromkeys(rank_list)


In [5]:
# cleaned up dict
rank_dict = {'3rd Best': 3,
 '4st': 4,
 '1st': 1,
 '3': 3,
 '4th Best': 4,
 '3st': 3,
 '2': 2,
 '2st': 2,
 '5st': 5,
 'Best': 1,
 '6th Best': 6,
 '6st': 6,
 '6': 6,
 '1': 1,
 '5th Best': 5,
 '2nd Best': 2,
 '4': 4,
 '5': 5}

# apply the dict with map
our_messy_metadata['Ranking_Cleaned'] = our_messy_metadata['Personal Rank'].map(rank_dict)



### Names

In [6]:
# the unique values
unique_names = our_messy_metadata['Your Name'].unique()
unique_names
# make a list
name_list = list(unique_names)
# dict
name_dict = {}
name_dict = name_dict.fromkeys(name_list)


In [7]:
name_dict = {'Natalie': "Queenan, Natalie",
 'cameron_neville': "Neville, Cameron",
 'isabella_law': "Law, Isabella",
 'Maryam Imran Siddiqui': "Siddiqui, Maryam Imran",
 'ethan_rezer': "Rezer, Ethan",
 'Andrew Michael Kostin': "Kostin, Andrew Michael",
 'Natalie ': "Queenan, Natalie",
 'Erin': "Monroe, Erin",
 'Max': "Hyun, Max",
 'Leo Ni': "Ni, Leo",
 'Hannah Grace Nosowsky Govert': "Govert, Hannah Grace Nosowsky",
 'miles_fagan': "Fagan, Miles",
 'Hannah Grace Nosowksy Govert': "Govert, Hannah Grace Nosowsky"}


our_messy_metadata['Names_Cleaned'] = our_messy_metadata['Your Name'].map(name_dict)



### Teams

In [8]:
team_list = list(our_messy_metadata['Team/Group Name'].unique())
team_dict = {}

team_dict = team_dict.fromkeys(team_list)
team_dict = {'Phlurp, Purple': "Purple Phlurp",
 'Greenmachine': "Green Machine",
 'Orange Crush': "Orange Crush",
 'Orange Crush ': "Orange Crush"}


our_messy_metadata['Teams_Cleaned'] = our_messy_metadata['Team/Group Name'].map(team_dict)


### Class

In [9]:
class_list = list(our_messy_metadata['Your Graduating Class'].unique())
class_dict = {}

class_dict = class_dict.fromkeys(class_list)
class_dict = {'Class o’ ‘28': 2028,
 '29': 2029,
 '27': 2027,
 'Blackbird': 2026,
 "Class o' '29": 2029,
 '2026': 2026,
 '2029': 2029,
 "Class 0' '29": 2029,
 '26': 2026,
 '‘29': 2029}


our_messy_metadata['Class_Cleaned'] = our_messy_metadata['Your Graduating Class'].map(class_dict)



### Most Likely Context

- We will need to do this more than once, since some terms are combinations!

In [23]:
most_likely_list = list(our_messy_metadata['Most Likely Context(s)'].unique())
most_likely_dict = {}

most_likely_dict = most_likely_dict.fromkeys(most_likely_list)
most_likely_dict = {'being_bored_in_class studying driving': 'being_bored_in_class studying driving',
 "Walkin'": 'walk',
 "Cryin'": 'emotion',
 'walking/driving': 'walking/driving',
 'walking driving studying': 'walking driving studying',
 "Drivin'": 'drive',
 'shopping/studying/walking/sitting': 'shopping/studying/walking/sitting',
 "weepin'": 'emotion',
 'studying/watching a live performance/waiting in an elevator': 'studying/watching a live performance/waiting in an elevator',
 'dancing watching_a_live_performance': 'dancing watching_a_live_performance',
 'get_off_my_lawn driving': 'get_off_my_lawn driving',
 "Dancin'": 'dance',
 'driving': 'drive',
 "Studyin'": 'study',
 'working out/being bored in class': 'working out/being bored in class',
 "live performin'": 'perform',
 'walking/watching a live performance/waiting in an elevator': 'walking/watching a live performance/waiting in an elevator',
 "Chillin'": 'relax',
 'being_bored_in_class studying': 'being_bored_in_class studying',
 'watching_a_live_performance': 'listen',
 'driving studying': 'driving studying',
 'working out': 'exercize',
 'walking/waiting in an elevator/watching a live performance': 'walking/waiting in an elevator/watching a live performance',
 "Performin'": 'perform',
 'watching a live performance': 'listen',
 'driving ': 'drive',
 "dancin'": 'dance',
 'walking': 'walk',
 'waiting_in_an_elevator': 'wait',
 'walking/studying': 'walking/studying',
 'DaNcInG': 'dance',
 'walking being_sad': 'walking being_sad',
 'driving/being bored in class': 'driving/being bored in class',
 'going out': 'explore',
 'being bored in class/walking': 'being bored in class/walking',
 'studying': 'study',
 'studying driving dancing': 'studying driving dancing',
 'dancing': 'dance',
 "bein' bored in class": 'boredom',
 'waiting in an elevator': 'wait',
 'driving being_bored_in_class': 'driving being_bored_in_class',
 "Thinkin'": 'think',
 'walking studying': 'walking studying',
 "watchin' a live performance": 'attend',
 'driving/walking/being bored in class': 'driving/walking/being bored in class',
 'driving/dancing/walking': 'driving/dancing/walking',
 'PoNdErInG': 'think',
 'studying/walking/chilling': 'studying/walking/chilling',
 'walking/waiting in an elevator': 'walking/waiting in an elevator',
 'studying/walking': 'studying/walking',
 'ReMiNiScInG': 'think',
 'being bored in class': 'boredom',
 'StUdYiNg': 'study',
 'WaItInG': 'wait',
 'SlEePiNg': 'sleep'}


our_messy_metadata['Most Likely Context Clean'] = our_messy_metadata['Most Likely Context(s)'].map(most_likely_dict).str.split(" ")

our_messy_metadata.iloc[0]['Most Likely Context Clean'][0]


'being_bored_in_class'

### Least Likely Context

Some terms are complex:  we will need to repeat and split!

In [11]:
least_likely_list = list(our_messy_metadata['Least Likely Contexts'].unique())
least_likely_dict = {}

least_likely_dict = least_likely_dict.fromkeys(least_likely_list)
least_likely_dict = {'watching_a_live_performance working_out': 'watching_a_live_performance working_out',
 "Dancin'": 'dance',
 "Moshin'": 'dance',
 'studying': 'study',
 'waiting_in_an_elevator': 'waiting',
 'dancing': 'dance',
 "workin' out": 'exercize',
 "Ravin'": 'dance',
 'working out': 'exercize',
 "Workin' Out": 'exercize',
 'working_out driving': 'working_out driving',
 "Studyin'": 'study',
 'studying/watching a live performance': 'studying/watching a live performance',
 "ravin'": 'dance',
 "Jammin'": 'dance',
 'working_out': 'exercize',
 'dancing/studying': 'dancing/studying',
 "Groovin'": 'dance',
 'working out/dancing': 'working out/dancing',
 "studyin'": 'study',
 "Clubbin'": 'dance',
 'dancing working_out driving': 'dancing working_out driving',
 'CrYiNg': 'feeling',
 "Sleepin'": 'sleep',
 'working out/waiting in an elevator': 'working out/waiting in an elevator',
 'sleeping': 'sleep',
 'working out/driving': 'working out/driving',
 'walking being_bored_in_class': 'walking being_bored_in_class',
 'being_bored_in_class': 'class',
 "sleepin'": 'sleep',
 'studying/dancing': 'studying/dancing',
 'waiting_in_an_elevator ': 'waiting',
 "waitin' in an elevator": 'waiting',
 'waiting in an elevator': 'waiting',
 'ExErCiSiNg': 'exercize',
 'crying': 'feeling',
 'dancing/working out': 'dancing/working out',
 'BoOgYiNg': 'dance',
 'SpOrTiNg': 'exercize',
 'RuNnInG': 'exercize',
 'DrIvInG': 'drive'}

our_messy_metadata['Least Likely Contexts Clean'] = our_messy_metadata['Least Likely Contexts'].map(least_likely_dict).str.split(' ')





### Genres

- This is quite complex--see various split, replace and strip
- We will also need to 'explode'

In [27]:
genre_list = list(our_messy_metadata['Genre Tags'].unique())
genre_list



['{soft rock} {indie rock} {rain}',
 'baroque.pop | alternative.rock',
 'acoustic.rock | british.invasion | classic.rock',
 'Rock',
 '{emo rap} {plugg}',
 'chamber.pop | folk',
 'mallsoft/blues rock',
 'folk.rock | acoustic.rock',
 'Soft Rock/Chill Guitar/Mallsoft',
 'pop | psychedelic.rock',
 '{candy pop} {power metal}',
 '{creepy rock} {weirdo} {alt rock}',
 'pop | psychedelia',
 '{rock} {sad}',
 'folk.rock | rock',
 'Steel Guitar/Futuristic Swag',
 'hippie | hopecore | classic.rock',
 'Psychedelic rock, classic rock',
 'chamber.pop | folk.rock',
 '{lonely} {classical_rock} {vocal_rock}',
 '{classic rock} {folk}',
 '{pop emo} {turkish alt pop}',
 'rock.and.roll | easy.listening',
 'Beatlesque/Soft Rock/Canadian Stoner Rock',
 'baroque.pop | art.rock',
 'Orchestral Soundtrack/Instrumental Rock',
 '{electro rock} {psychadelic} {long_build}',
 'merseybeat | rock.and.roll | r&b',
 'baroque.pop | soft.rock',
 '{classic rock} {happy}',
 '{irish singer-songwriter} {czsk hip hop}',
 'Contemp

In [26]:


our_messy_metadata['Genre Tags Clean'] = our_messy_metadata['Genre Tags'].str.replace('{', '').str.split(r'[|}/]')

our_messy_metadata.iloc[1]['Genre Tags Clean']

['baroque.pop ', ' alternative.rock']

In [14]:
our_messy_metadata.to_pickle('our_clean_metadata_stage_1.pkl')

### Clean Up Columns


In [15]:
our_messy_metadata.columns

Index(['Timestamp', 'Song Title', 'Your Name', 'Your Graduating Class',
       'Team/Group Name', 'Personal Rank', 'Genre Tags',
       'Most Likely Context(s)', 'Least Likely Contexts', 'Ranking_Cleaned',
       'Names_Cleaned', 'Teams_Cleaned', 'Class_Cleaned',
       'Most Likely Context Clean', 'Least Likely Contexts Clean',
       'Genre Tags Clean'],
      dtype='object')

In [16]:
cols = ['Song Title', 'Names_Cleaned', 'Ranking_Cleaned',
       'Class_Cleaned', 'Most Likely Context Clean',
       'Least Likely Contexts Clean', 'Genre Tags Clean']

clean_stage_one = our_messy_metadata[cols]
clean_stage_one

,Song Title,Names_Cleaned,Ranking_Cleaned,Class_Cleaned,Most Likely Context Clean,Least Likely Contexts Clean,Genre Tags Clean
0,Blackbird,"Queenan, Natalie",3,2028,"[being_bored_in_class, studying, driving]","[watching_a_live_performance, working_out]","[soft rock, indie rock, rain, ]"
1,Eleanor Rigby,"Neville, Cameron",4,2029,[walk],[dance],"[baroque.pop , alternative.rock]"
2,Yesterday,"Law, Isabella",1,2027,[emotion],[dance],"[acoustic.rock , british.invasion , classic...."
3,A Day in the Life,"Siddiqui, Maryam Imran",3,2027,[walking/driving],[study],[Rock]
4,Blackbird,"Queenan, Natalie",4,2028,"[walking, driving, studying]",[waiting],"[emo rap, plugg, ]"
...,...,...,...,...,...,...,...
61,Good Night,"Govert, Hannah Grace Nosowsky",5,2026,[boredom],"[dancing/working, out]","[UK Contemporary R&B, Rock]"
62,sea of holes,"Govert, Hannah Grace Nosowsky",6,2026,[wait],[study],"[Psychedelic Rock, British Soundtrack]"
63,All You Need Is Love,"Fagan, Miles",3,2029,[study],[exercize],"[psychedelic.rock , experimental.rock]"
64,Strawberry Fields Forever,"Fagan, Miles",4,2029,[wait],[exercize],"[psychedelic.rock , progressive.pop]"


In [18]:
#
pd.read_pickle('our_clean_metadata_stage_1.pkl')

,Timestamp,Song Title,Your Name,Your Graduating Class,Team/Group Name,Personal Rank,Genre Tags,Most Likely Context(s),Least Likely Contexts,Ranking_Cleaned,Names_Cleaned,Teams_Cleaned,Class_Cleaned,Most Likely Context Clean,Least Likely Contexts Clean,Genre Tags Clean
0,9/17/2025 15:21:26,Blackbird,Natalie,Class o’ ‘28,"Phlurp, Purple",3rd Best,{soft rock} {indie rock} {rain},being_bored_in_class studying driving,watching_a_live_performance working_out,3,"Queenan, Natalie",Purple Phlurp,2028,"[being_bored_in_class, studying, driving]","[watching_a_live_performance, working_out]","[soft rock, indie rock, rain, ]"
1,9/17/2025 15:22:00,Eleanor Rigby,cameron_neville,29,Greenmachine,4st,baroque.pop | alternative.rock,Walkin',Dancin',4,"Neville, Cameron",Green Machine,2029,[walk],[dance],"[baroque.pop , alternative.rock]"
2,9/17/2025 15:22:33,Yesterday,isabella_law,27,Greenmachine,1st,acoustic.rock | british.invasion | classic.rock,Cryin',Moshin',1,"Law, Isabella",Green Machine,2027,[emotion],[dance],"[acoustic.rock , british.invasion , classic...."
3,9/17/2025 15:23:31,A Day in the Life,Maryam Imran Siddiqui,27,Orange Crush,3,Rock,walking/driving,studying,3,"Siddiqui, Maryam Imran",Orange Crush,2027,[walking/driving],[study],[Rock]
4,9/17/2025 15:23:54,Blackbird,Natalie,Class o’ ‘28,"Phlurp, Purple",4th Best,{emo rap} {plugg},walking driving studying,waiting_in_an_elevator,4,"Queenan, Natalie",Purple Phlurp,2028,"[walking, driving, studying]",[waiting],"[emo rap, plugg, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,9/17/2025 15:39:06,Good Night,Hannah Grace Nosowsky Govert,26,Orange Crush,5,UK Contemporary R&B/Rock,being bored in class,dancing/working out,5,"Govert, Hannah Grace Nosowsky",Orange Crush,2026,[boredom],"[dancing/working, out]","[UK Contemporary R&B, Rock]"
62,9/17/2025 15:40:57,sea of holes,Hannah Grace Nosowsky Govert,26,Orange Crush,6,Psychedelic Rock/British Soundtrack,waiting in an elevator,studying,6,"Govert, Hannah Grace Nosowsky",Orange Crush,2026,[wait],[study],"[Psychedelic Rock, British Soundtrack]"
63,9/17/2025 16:20:37,All You Need Is Love,miles_fagan,‘29,Greenmachine,3st,psychedelic.rock | experimental.rock,StUdYiNg,SpOrTiNg,3,"Fagan, Miles",Green Machine,2029,[study],[exercize],"[psychedelic.rock , experimental.rock]"
64,9/17/2025 16:23:12,Strawberry Fields Forever,miles_fagan,‘29,Greenmachine,4st,psychedelic.rock | progressive.pop,WaItInG,RuNnInG,4,"Fagan, Miles",Green Machine,2029,[wait],[exercize],"[psychedelic.rock , progressive.pop]"


# 3. Interpretation

What did you learn from the results? How were your expectations confirmed (or not)?

What limitations did the data or your methods impose on your capacity to answer the questions you originally posed?

How do your results change your views of the readings or ideas that motivated your work it the first place? How do your results shape your listening experience of the music you considered? What next steps do you imagine for future work?

# 4. Bibliography

A list of the sources consulted and cited during your work, including class readings or code resources, Spotify lists, etc. See the Notebook Style Guide and Music Style Guide for details about how to cite books, articles, internet resources, etc.

In [28]:
beatles_spotify_pkl = 'https://raw.githubusercontent.com/RichardFreedman/Encoding_Music/main/02_Lab_Data/Beatles/beatles_data.pkl'

# Import to a Pandas dataframe
beatles_spotify = pd.read_pickle(beatles_spotify_pkl)

In [29]:
beatles_spotify

,title,album,year,spotify_id,songwriter,lead_vocal,genre,billboard_weeks,danceability,energy,...,instrumentalness,liveness,valence,loudness,mode,key,tempo,duration_ms,time_signature,song_count
0,a day in the life,Sgt. Pepper's Lonely Hearts Club Band,1967,"0hKRSZhUGEhKU6aNSPBACZ""",Lennon and McCartney,Lennon and McCartney,psychedelic rock,0,0.364,0.457,...,0.000106,0.9220,0.175,-14.162,0,4,163.219,337413,4,1
1,a day in the life,Sgt. Pepper's Lonely Hearts Club Band,1967,"0hKRSZhUGEhKU6aNSPBACZ""",Lennon and McCartney,Lennon and McCartney,art rock,0,0.364,0.457,...,0.000106,0.9220,0.175,-14.162,0,4,163.219,337413,4,1
2,a day in the life,Sgt. Pepper's Lonely Hearts Club Band,1967,"0hKRSZhUGEhKU6aNSPBACZ""",Lennon and McCartney,Lennon and McCartney,pop rock,0,0.364,0.457,...,0.000106,0.9220,0.175,-14.162,0,4,163.219,337413,4,1
3,a hard day's night,A Hard Day's Night,1964,5J2CHimS7dWYMImCHkEFaJ,Lennon,"Lennon, with McCartney",rock,8,0.590,0.805,...,0.000000,0.0996,0.797,-6.481,1,0,138.514,154200,4,1
4,a hard day's night,A Hard Day's Night,1964,5J2CHimS7dWYMImCHkEFaJ,Lennon,"Lennon, with McCartney",electronic,8,0.590,0.805,...,0.000000,0.0996,0.797,-6.481,1,0,138.514,154200,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,you never give me your money,Abbey Road,1969,1jOLTO379yIu9aMnCkpMQl,McCartney,McCartney,pop rock,0,0.335,0.416,...,0.136000,0.1160,0.223,-11.051,1,0,174.464,242973,3,1
295,your mother should know,Magical Mystery Tour,1967,1dxbAIfCASqv6jix2R1Taj,McCartney,McCartney,music hall,0,0.698,0.293,...,0.010600,0.0936,0.724,-11.361,1,0,117.613,148413,4,1
296,your mother should know,Magical Mystery Tour,1967,1dxbAIfCASqv6jix2R1Taj,McCartney,McCartney,vaudeville rock,0,0.698,0.293,...,0.010600,0.0936,0.724,-11.361,1,0,117.613,148413,4,1
297,your mother should know,Magical Mystery Tour,1967,1dxbAIfCASqv6jix2R1Taj,McCartney,McCartney,psychedelic pop,0,0.698,0.293,...,0.010600,0.0936,0.724,-11.361,1,0,117.613,148413,4,1


In [30]:
beatles_billboard_csv = 'https://raw.githubusercontent.com/inteligentni/Class-05-Feature-engineering/master/The%20Beatles%20songs%20dataset%2C%20v1%2C%20no%20NAs.csv'

beatles_billboard = pd.read_csv(beatles_billboard_csv)
beatles_billboard

,Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
0,12-Bar Original,1965,Anthology 2,174,0,Blues,"Lennon, McCartney, Harrison and Starkey",NaN,-1
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1
2,A Hard Day's Night,1964,UK: A Hard Day's Night US: 1962-1966,152,35,"Rock, Electronic, Pop/Rock",Lennon,"Lennon, with McCartney",8
3,A Shot of Rhythm and Blues,1963,Live at the BBC,104,0,"R&B, Pop/Rock",Thompson,Lennon,-1
4,A Taste of Honey,1963,UK: Please Please Me US: The Early Beatles,163,29,"Pop/Rock, Jazz, Stage&Screen","Scott, Marlow",McCartney,-1
...,...,...,...,...,...,...,...,...,...
305,You're Going to Lose That Girl,1965,Help!,140,6,"Rock, Pop/Rock",Lennon,Lennon,-1
306,You've Got to Hide Your Love Away,1965,Help!,131,12,FolkPop/Rock,Lennon,Lennon,-1
307,You've Really Got a Hold on Me,1963,UK: With the Beatles US: The Beatles Second Album,182,2,"Soul, Pop/Rock",Robinson,Lennon and Harrison,-1
308,Young Blood,1963,Live at the BBC,116,0,Pop/Rock,"Leiber, Stoller",Harrison,-1


In [35]:
beatles_billboard.iloc[1]['Genre Split']

['Psychedelic Rock', 'Art Rock', 'Pop/Rock']

In [39]:
beatles_billboard['Genre Split'] = beatles_billboard['Genre'].str.split(', ')
beatles_billboard_exploded = beatles_billboard.explode('Genre Split')

In [40]:
beatles_billboard_exploded.head()

,Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard,Genre Split
0,12-Bar Original,1965,Anthology 2,174,0,Blues,"Lennon, McCartney, Harrison and Starkey",NaN,-1,Blues
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1,Psychedelic Rock
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1,Art Rock
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1,Pop/Rock
2,A Hard Day's Night,1964,UK: A Hard Day's Night US: 1962-1966,152,35,"Rock, Electronic, Pop/Rock",Lennon,"Lennon, with McCartney",8,Rock


In [51]:
beatles_spotify_csv = 'https://raw.githubusercontent.com/RichardFreedman/Encoding_Music/refs/heads/main/02_Lab_Data/Billboard%20and%20Spotify%20Data/Beatles_Spotify.csv'

beatles_spotify = pd.read_csv(beatles_spotify_csv)
beatles_spotify

,Unnamed: 0,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Album_Name
0,0,The Beatles,Get Back - Rooftop Performance / Take 1,4bVT9bpThKnkFtl1xeB2bh,0.628,0.524,2,-11.840,1,0.0773,0.099500,0.1950,0.2720,124.176,283373,4,Get Back (Rooftop Performance)
1,1,The Beatles,Get Back - Rooftop Performance / Take 2,4z3RNAUb0whdaLAudM9EHE,0.655,0.695,9,-9.648,1,0.0381,0.436000,0.1050,0.3300,125.572,204187,4,Get Back (Rooftop Performance)
2,2,The Beatles,Don't Let Me Down - Rooftop Performance / Take 1,6Wjbtcnv4Yw92luvE1Y8tW,0.592,0.513,6,-8.778,0,0.0322,0.000170,0.1820,0.7440,81.920,202467,4,Get Back (Rooftop Performance)
3,3,The Beatles,I've Got A Feeling - Rooftop Performance / Take 1,06rGbrn3Vhp6CeByhZboOT,0.392,0.629,2,-11.235,1,0.1720,0.006990,0.0847,0.3520,165.416,283720,4,Get Back (Rooftop Performance)
4,4,The Beatles,One After 909 - Rooftop Performance,4MBeqWnCdIMcEexCNbNqRt,0.521,0.870,4,-8.985,1,0.2370,0.000002,0.6220,0.7230,90.474,189200,4,Get Back (Rooftop Performance)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,540,The Beatles,Got To Get You Into My Life - Remastered 2009,3tGhRLgcCP6SIZU3tbGl7l,0.521,0.524,0,-6.136,1,0.0302,0.000009,0.0838,0.7110,136.679,149240,4,Revolver (Remastered)
541,541,The Beatles,Tomorrow Never Knows - Remastered 2009,00oZhqZIQfL9P5CjOP6JsO,0.380,0.829,5,-6.105,1,0.0405,0.002080,0.3420,0.0534,125.887,179547,4,Revolver (Remastered)
542,542,The Beatles,Two Of Us - 2021 Mix,2CEujT1Tuv3jIIjUyUvupg,0.692,0.756,7,-9.677,1,0.0402,0.000088,0.2190,0.8260,109.111,216480,4,Let It Be (Super Deluxe)
543,543,The Beatles,Dig A Pony - 2021 Mix,5pbQlOzqzTacF4wbzmWEeA,0.461,0.500,11,-8.023,0,0.0478,0.000010,0.2090,0.3930,118.377,234760,3,Let It Be (Super Deluxe)


In [52]:
# Choose our ID Vars, or identifiers - the column to keep fixed
id_vars = ['track_name', 'artist']

# Choose our value_vars, or columns to "unpivot" into a single "variable" column.
value_vars = ['danceability', 'energy', 'speechiness']

# Using melt
melted_feature_data = pd.melt(
    beatles_spotify, # Dataset
    id_vars=id_vars, # Pivot
    value_vars=value_vars # Values
).sort_values('track_name').reset_index(drop=True) # Sorting

In [53]:
melted_feature_data

,track_name,artist,variable,value
0,"1822! - Live At The BBC For ""Pop Go The Beatle...",The Beatles,danceability,0.0000
1,"1822! - Live At The BBC For ""Pop Go The Beatle...",The Beatles,speechiness,0.0000
2,"1822! - Live At The BBC For ""Pop Go The Beatle...",The Beatles,energy,0.0923
3,A Day In The Life - Hummed Last Chord / Takes ...,The Beatles,energy,0.2260
4,A Day In The Life - Hummed Last Chord / Takes ...,The Beatles,danceability,0.4230
...,...,...,...,...
1630,"Young Blood - Live At The BBC For ""Pop Go The ...",The Beatles,energy,0.6150
1631,"Young Blood - Live At The BBC For ""Pop Go The ...",The Beatles,speechiness,0.0358
1632,Your Mother Should Know - Remastered 2009,The Beatles,speechiness,0.0332
1633,Your Mother Should Know - Remastered 2009,The Beatles,energy,0.2930
